# Modelo Recomendación
Se creará un modelo de recomendación de películas, basado en el dataset recibido, la salida retornará una lista de 5 películas similares recomendadas por el modelo según sus características, dado un título de una película perteneciente al dataset

In [99]:
import pandas as pd

## Preprocesamiento de datos

Carga del dataset

In [100]:
df_premodel = pd.read_parquet('https://github.com/alejocampos1/Henry_PI1_Alejandro-Campos/raw/main/Datasets/Datasets_Limpios/Parquet/movies_tomodel.parquet')
df_genres = pd.read_parquet('https://github.com/alejocampos1/Henry_PI1_Alejandro-Campos/raw/main/Datasets/Datasets_Limpios/Parquet/genres.parquet')
df_prodcompanies = pd.read_parquet("https://github.com/alejocampos1/Henry_PI1_Alejandro-Campos/raw/main/Datasets/Datasets_Limpios/Parquet/prodcompanies.parquet")

In [101]:
df_premodel.head(2)

,id,title,genres,overview,runtime,production_companies,original_language,spoken_languages,production_countries,popularity,vote_average,vote_count,release_year,weighted_rating
0,862,Toy Story,"[16, 35, 10751]","Led by Woody, Andy's toys live happily in his ...",81.0,[3],en,[en],[US],21.946943,7.7,5415.0,1995,7.68
1,8844,Jumanji,"[12, 14, 10751]",When siblings Judy and Peter discover an encha...,104.0,"[559, 2550, 10201]",en,"[en, fr]",[US],17.015539,6.9,2413.0,1995,6.87


## One-hot label encoding para la columna 'géneros'

Primero, se cambiará el 'id' de género en la columna por su nombre real

In [102]:
# Creamos un diccionario que relacione los IDs de los géneros con sus nombres
generos_dict = dict(zip(df_genres['id'], df_genres['name']))

# Crear una nueva columna en df_premodel con los nombres de los géneros en lugar de los IDs
df_premodel['genres_names'] = df_premodel['genres'].apply(
    lambda x: [generos_dict[item_id] for item_id in x]
)

# Eliminar columna duplicada
df_premodel.drop(columns=['genres'], inplace=True)

df_premodel.head(2)

,id,title,overview,runtime,production_companies,original_language,spoken_languages,production_countries,popularity,vote_average,vote_count,release_year,weighted_rating,genres_names
0,862,Toy Story,"Led by Woody, Andy's toys live happily in his ...",81.0,[3],en,[en],[US],21.946943,7.7,5415.0,1995,7.68,"[Animation, Comedy, Family]"
1,8844,Jumanji,When siblings Judy and Peter discover an encha...,104.0,"[559, 2550, 10201]",en,"[en, fr]",[US],17.015539,6.9,2413.0,1995,6.87,"[Adventure, Fantasy, Family]"


Creación del one-hot label enconding

In [103]:
# Expandir las listas de géneros en filas individuales
df_generos_encoded = df_premodel[['id', 'genres_names']].explode('genres_names')

# Get_dummies para crear las columnas one hot
df_generos_encoded = pd.get_dummies(df_generos_encoded, columns=['genres_names'], prefix='', prefix_sep='')

# Nuevo Dataframe agrupado

df_generos_encoded = df_generos_encoded.groupby('id').sum().reset_index()

# Hacer un merge de df_premodel con df_generos_encoded usando la columna 'id'
df_premodel = pd.merge(df_premodel, df_generos_encoded, on='id', how='left')

# Eliminar columna 'genres_names'
df_premodel.drop(columns=['genres_names'], inplace=True)

# Verificar el resultado
df_premodel.head(2)


,id,title,overview,runtime,production_companies,original_language,spoken_languages,production_countries,popularity,vote_average,...,Horror,Music,Mystery,No Data,Romance,Science Fiction,TV Movie,Thriller,War,Western
0,862,Toy Story,"Led by Woody, Andy's toys live happily in his ...",81.0,[3],en,[en],[US],21.946943,7.7,...,0,0,0,0,0,0,0,0,0,0
1,8844,Jumanji,When siblings Judy and Peter discover an encha...,104.0,"[559, 2550, 10201]",en,"[en, fr]",[US],17.015539,6.9,...,0,0,0,0,0,0,0,0,0,0


## One-hot label encoding para la columna 'production_companies'

Para las compañías productoras, utilizaremos solamente las 100 más populares, las demás, será agrupadas en la categoría 'OTROS'

In [104]:
# Filtrar las primeras 100 productoras por popularidad
df_top100_prod = (
    df_premodel[['production_companies', 'popularity']]
    .explode('production_companies')  # Descomponer la columna de listas
    .groupby('production_companies', as_index=False)['popularity'].sum()  # Agrupar por el id de productora y sumar popularidades
    .merge(df_prodcompanies[['id', 'name']], left_on='production_companies', right_on='id')  # Hacer el merge con nombres de productoras
    [['id', 'name', 'popularity']]  # Seleccionar las columnas finales, incluyendo los IDs de las productoras
    .sort_values(by='popularity', ascending=False)  # Ordenar por popularidad de forma descendente
    .reset_index(drop=True)  # Restablecer el índice
    .iloc[:100]  # Seleccionar solo las primeras 100 filas
)

In [105]:
df_top100_prod.head(1)

,id,name,popularity
0,6194,Warner Bros.,7717.437823


In [106]:
# Crear un set con los IDs de las top 100 productoras
top100_prod_ids = set(df_top100_prod['id'])

# Crear un diccionario que relacione los IDs de las compañías productoras con sus nombres
prodcomp_dict = dict(zip(df_prodcompanies['id'], df_prodcompanies['name']))

# Crear una nueva columna en df_premodel con los nombres de las compañías productoras en lugar de los IDs
df_premodel['production_companies_new'] = df_premodel['production_companies'].apply(
    lambda x: ["Other Prod Company" if item_id not in top100_prod_ids else prodcomp_dict.get(item_id) for item_id in x]
)

# Verificar el resultado
df_premodel['production_companies_new']

0                                [Pixar Animation Studios]
1        [TriStar Pictures, Other Prod Company, Other P...
2                       [Warner Bros., Other Prod Company]
3                 [Twentieth Century Fox Film Corporation]
4                [Other Prod Company, Touchstone Pictures]
                               ...                        
43231             [Other Prod Company, Other Prod Company]
43232                                 [Other Prod Company]
43233    [Other Prod Company, Working Title Films, Othe...
43234                                 [Other Prod Company]
43235                                                   []
Name: production_companies_new, Length: 43236, dtype: object

In [107]:
# Expandir las listas de géneros en filas individuales
df_prodcompany_encoded = df_premodel[['id', 'production_companies_new']].explode('production_companies_new')

# Get_dummies para crear las columnas one hot
df_prodcompany_encoded = pd.get_dummies(df_prodcompany_encoded, columns=['production_companies_new'], prefix='', prefix_sep='')

# Nuevo Dataframe agrupado
df_prodcompany_encoded = df_prodcompany_encoded.groupby('id').sum().reset_index()

# Hacer un merge de df_premodel con df_prodcompany_encoded usando la columna 'id'
df_premodel = pd.merge(df_premodel, df_prodcompany_encoded, on='id', how='left')

# Eliminar columnas 'production_companies', 'production_companies_new'
df_premodel.drop(columns=['production_companies', 'production_companies_new' ], inplace=True)

# Corregir datos en 'Other Prod Company'
for index, row in df_premodel.iterrows():
    if row['Other Prod Company'] > 1:
        df_premodel.at[index, 'Other Prod Company'] = 1

# Verificar el resultado
df_premodel.head(1)

,id,title,overview,runtime,original_language,spoken_languages,production_countries,popularity,vote_average,vote_count,...,Vertigo Entertainment,Village Roadshow Pictures,Walt Disney Animation Studios,Walt Disney Pictures,Walt Disney Productions,Wanda Pictures,Warner Bros.,Warner Bros. Animation,Wild Bunch,Working Title Films
0,862,Toy Story,"Led by Woody, Andy's toys live happily in his ...",81.0,en,[en],[US],21.946943,7.7,5415.0,...,0,0,0,0,0,0,0,0,0,0
